<a href="https://colab.research.google.com/github/Dev-hottae/PINC/blob/master/Data_Analysis/Summarize_KoGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gluonnlp mxnet tqdm
!pip install sentencepiece==0.1.85
!pip install torch==1.6.0
!pip install transformers==2.1.1
!pip install kss nltk

     |████████████████████████████████| 348kB 4.6MB/s 
     |████████████████████████████████| 55.0MB 52kB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588534 sha256=8791ed9106528cfe3ace400aa00a2ce4626635e45ac10971463c5a203c933776
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 1.0MB 4.7MB/s 
     |████████████████████████████████| 317kB 4.5MB/s 
     |████████████████████████████████| 890kB 6.6MB/s 
     |████████████████████████████████| 133kB 14.7MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 6.7MB 17.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d8696595be567e805ead6eb16b772cf88

In [2]:
!pip install git+https://github.com/SKT-AI/KoGPT2.git

  Cloning https://github.com/SKT-AI/KoGPT2.git to /tmp/pip-req-build-4d_aebiq
  Running command git clone -q https://github.com/SKT-AI/KoGPT2.git /tmp/pip-req-build-4d_aebiq
  Created wheel for kogpt2: filename=kogpt2-0.1.1-cp36-none-any.whl size=14054 sha256=22c2fd5bee29bffb2daf9a4318e13e8fd122999f3d048993408ba4daa885c238
  Stored in directory: /tmp/pip-ephem-wheel-cache-h2v2dkbz/wheels/3b/a2/30/432bb7490a2ea23a90049e6c5725f6acd7e925f1abfb3d7ddf
Successfully built kogpt2


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from gluonnlp.data import SentencepieceTokenizer 
import gluonnlp as nlp
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm, tqdm_notebook
from nltk.tokenize import sent_tokenize

In [5]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
from kogpt2.utils import download, tokenizer, get_tokenizer
from kogpt2.pytorch_kogpt2 import GPT2Config, GPT2LMHeadModel

In [6]:
#GPU 사용
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [7]:
#ctx= 'cuda'#'cuda' #'cpu' #학습 Device CPU or GPU. colab의 경우 GPU 사용
cachedir='~/kogpt2/' # KoGPT-2 모델 다운로드 경로
epoch = 50  # 학습 epoch
save_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/'
use_cuda = True # Colab내 GPU 사용을 위한 값

pytorch_kogpt2 = {
    'url':
    'https://kobert.blob.core.windows.net/models/kogpt2/pytorch/pytorch_kogpt2_676e9bcfa7.params',
    'fname': 'pytorch_kogpt2_676e9bcfa7.params',
    'chksum': '676e9bcfa7'
}
kogpt2_config = {
    "initializer_range": 0.02,
    "layer_norm_epsilon": 1e-05,
    "n_ctx": 1024,
    "n_embd": 768,
    "n_head": 12,
    "n_layer": 12,
    "n_positions": 1024,
    "vocab_size": 50000
}

In [8]:
# download model
model_info = pytorch_kogpt2
model_path = download(model_info['url'],
                       model_info['fname'],
                       model_info['chksum'],
                       cachedir=cachedir)
# download vocab
vocab_info = tokenizer
vocab_path = download(vocab_info['url'],
                       vocab_info['fname'],
                       vocab_info['chksum'],
                       cachedir=cachedir)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
# KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
# model_path로부터 다운로드 받은 내용을 load_state_dict으로 업로드
model.load_state_dict(torch.load(model_path))

# device = torch.device(ctx)
model = model.to(device)

# kogpt2model.eval()
# 추가로 학습하기 위해 .train() 사용
model.train()
vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_path,
                                                     mask_token=None,
                                                     sep_token=None,
                                                     cls_token=None,
                                                     unknown_token='<unk>',
                                                     padding_token='<pad>',
                                                     bos_token='<s>',
                                                     eos_token='</s>')

In [10]:
tok_path = get_tokenizer()
vocab = vocab_b_obj
sentencepieceTokenizer = SentencepieceTokenizer(tok_path)

using cached model


In [11]:
data_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/DataSet/Topic_keywords.csv'

In [12]:
news_data = pd.read_csv(data_path, encoding='utf-8')
news_data.head()

,Date,Topic_keywords,Text
0,2006-06-27,삼성전자 하이닉스 메모리 긍정 증권 업체 분기 미래 대비 수준,하이닉스반도체가 블록딜을 마치자마자 주가가 급등했다. 하이닉스 주가는 26일 유가증...
1,2006-06-27,하이닉스 발행 주가 신주 매각 외국 이번 만주 권단 주식,코스피지수가 엿새 만에 1240선을 회복했다. 27일 코스피지수는 전날 미국 증시 ...
2,2006-06-27,외국인 매도 상승 종목 이날 순매도 하락 소식 사흘 관련,코스피지수가 하락 하루 만에 반등하며 1240선에 다가섰다. 26일 코스피지수는 전...
3,2006-06-27,지수 이날 상승 종목 시장 프로그램 순매도 매수세 기록 금융,세계시장에 나갔을 때 '코리아 디스카운트'가 이 정도로 심할 줄은 몰랐습니다. 그러...
4,2006-06-27,창업 시장 원자 벤처 미국 대표 현미경 장비 매출 세계,"일본 닛케이225지수는 직전 저점 대비 6.7% 상승, 20일이동평균선을 회복했다...."


In [13]:
# 학습, 테스트 데이터 분리
train_data = news_data.sample(frac=0.8, random_state=2020)
test_data = news_data.drop(train_data.index)
len(train_data), len(test_data)

(80, 20)

In [14]:
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.drop(['index'], axis=1)
test_data = test_data.dropna()
test_data = test_data.reset_index()
test_data = test_data.drop(['index'], axis=1)
len(train_data), len(test_data)

(80, 20)

In [15]:
dataset_train = [] # 라벨은 0부터 순서대로 입력해야함
dataset_test = []
for i in tqdm(range(len(train_data))):
    dataset_train.append([train_data['Topic_keywords'][i], train_data['Text'][i]]) # 해당 리스트 형태를 맞춰야 학습 가능
for i in tqdm(range(len(test_data))):
    dataset_test.append([test_data['Topic_keywords'][i], test_data['Text'][i]])

dataset_train[:5]

100%|██████████| 20/20 [00:00<00:00, 5066.50it/s]


[['종목 기록 가운데 하락 지수 시스템 순매도 투자 푸드 사흘',
  ' 29일 오전 아시아 주요국 주식시장이 일제히 상승세를 타고 있다. 최근 미 연준의 금리인상 우려로 약세를 보였던 시장이 막상 정책 발표를 코 앞에 두고 안정된 흐름을 보이고 있다. 이에 대해 BT파이낸셜의 트로이 앵거스 연구원은 금리 인상 가능성이 이미 시장에 반영됐고 연준이 성명서 내용을 변경할 것 같지도 않다 며 예상을 뛰어넘는 일이 없는 한, 주식시장이 크게 동요하지는 않을 것 이라고 평가했다. 일본의 닛케이225 지수 는 전일 대비 1.19% 오른 1만5063.48로 오전장을 마감, 한 주간 가장 큰 폭으로 상승했다. 토픽스 지수 역시 1.19% 상승한 1545.75를 기록했다. 그동안 낙폭이 컸던 수출주들이 많이 올랐다. NEC와 엘피다메모리가 각각 3.3%, 3.61% 올랐고, 캐논과 마쓰시타 전자, 소니, 무라타 매뉴팩쳐링, 후지쓰 등이 일제히 2% 이상 상승했다. 원유와 구리를 비롯한 상품가격이 상승함에 따라 관련주에도 매수세가 몰렸다. 신일본제철이 1.44% 올랐고, JFE홀딩스는 2.38% 상승했다. 스미토모 금속공업과 고베제강도 각각 0.44%, 1.79%의 상승률을 기록했다.'],
 ['진도 결정 보증 에프 채무 경제 금지 무단 배포 전재',
  '코스닥시장이 해외증시 상승의 영향으로 강세를 보이고 있다. 27일 오후 2시20분 현재 코스닥시장은 전날대비 4.73포인트 오른 565.65을 기록하고 있다. 거래대금은 1조1383억원. 코스닥시장이 전날 미국주식시장 상승의 영향으로 외국인들이 매수에 나서면서 큰 폭으로 오르고 있다. 이 시각 현재 코스닥시장에서 개인과 외국인은 각각 173억, 31억원 매수우위를 보이고 있다. 기관은 117억원어치를 순매도하고 있다. 장 초반 소형주가 강세를 보였던 것과 달리 오후 들어 대형주가 큰 폭으로 오르고 있다. 코스닥100지수는 전날대비 1.09% 오르고 있다. 코스닥중형300지수, 코스닥소형지수는 각각 0.65%, 0.44% 

In [16]:
class GPTDataset(Dataset):
  def __init__(self, data_file, vocab, tokenizer):
    self.data =[]
    self.vocab = vocab
    self.tokenizer = tokenizer

    for i in data_file:
        toeknized_line = self.tokenizer(i[1])
        if len(toeknized_line) <= 1020:
            index_of_words = [vocab.bos_token] + toeknized_line + [vocab.padding_token] * (1020 - len(toeknized_line)) + [vocab.eos_token]
            self.data.append(vocab(index_of_words))

  def __len__(self):
    return len(self.data)
  def __getitem__(self,index):
    item = self.data[index]

    return item

In [17]:
batch_size = 1
news_dataset = GPTDataset(dataset_train, vocab, sentencepieceTokenizer)
news_data_loader = DataLoader(news_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=3)

In [18]:
learning_rate = 1e-5
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
epoch = 1
for epoch in range(epoch):
  count = 0
  avg_loss = (0.0, 0.0)
  for data in tqdm(news_data_loader):

    optimizer.zero_grad()
    data = torch.stack(data)
    data = data.transpose(1,0)
    data = data.to(device)
    model = model.to(device)

    outputs = model(data, labels=data)
    loss, logits = outputs[:2]
    loss = loss.to(device)
    loss.backward()
    avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
    optimizer.step()
    count+=1

    print('epoch no.{0} train no.{1}  loss = {2:.5f} avg_loss = {3:.5f}' . format(epoch, count, loss, avg_loss[0] / avg_loss[1]))

    # 추론 및 학습 재개를 위한 일반 체크포인트 저장하기
    # if count%2 == 0:
    torch.save({
        'epoch': epoch,
        'train_no': count,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':loss
      }, save_path+'narrativeKoGPT2_checkpoint.tar')

    count += 1

  0%|          | 0/80 [00:00<?, ?it/s]Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_pro

KeyboardInterrupt: ignored

In [24]:
load_path = '/content/drive/My Drive/머신러닝/팀 프로젝트/06. AI를 이용한 금융 보고서/Data_Analysis/checkpoint/narrativeKoGPT2_checkpoint.tar'
# 저장한 Checkpoint 불러오기
checkpoint = torch.load(load_path, map_location=device)

# # KoGPT-2 언어 모델 학습을 위한 GPT2LMHeadModel 선언
kogpt2model = GPT2LMHeadModel(config=GPT2Config.from_dict(kogpt2_config))
kogpt2model.load_state_dict(checkpoint['model_state_dict'])

kogpt2model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [38]:
import random

def top_p(logits, vocab, threshold = 0.9):
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    indexs = sorted_indices.tolist()

    sorted_softmax_logits = F.softmax(sorted_logits, dim=-1)
    cum_probs = torch.cumsum( sorted_softmax_logits, dim=-1)


    sorted_indices_to_remove = cum_probs > threshold
    top_p_index = 0

    # Top-p에 해당하는 index를 획득
    for i in range(len(sorted_indices_to_remove)):
      if sorted_indices_to_remove[i]== True:
        top_p_index = 0 if i==0 else i-1
        break

    # for i in range(top_p_index):
      # print('gen '+str(i)+': '+vocab.to_tokens(indexs[i]))

    rand_num = random.randint(0, top_p_index) # top-p 분포에서 랜덤 샘플링
    top_p_sample_num = indexs[rand_num]
    gen_word = vocab.to_tokens(top_p_sample_num)
    #print('selected token: '+gen_word+ ' softmax value:'+str(sorted_softmax_logits[rand_num]))

    return gen_word

In [41]:
sent = input('문장 입력: ')

toked = sentencepieceTokenizer(sent)
count = 0
output_size = 200 # 출력하고자 하는 토큰 갯수

while 1:
    input_ids = torch.tensor([vocab[vocab.bos_token],]  + vocab[toked]).unsqueeze(0)
    input_ids = input_ids.to(device)
    predicts = model(input_ids)
    pred = predicts[0]

    last_pred = pred.squeeze()[-1]
    # top_p 샘플링 방법
    # sampling.py를 통해 random, top-k, top-p 선택 가능.
    gen = top_p(last_pred, vocab, 0.9)
    # gen = sampling.top_k(last_pred, vocab, 5)

    if count>output_size:
        sent += gen.replace('▁', ' ')
        toked = sentencepieceTokenizer(sent)
        count =0
        break
    sent += gen.replace('▁', ' ')
    toked = sentencepieceTokenizer(sent)
    count += 1

문장 입력: 임호태 바보


In [42]:
for s in sent_tokenize(sent):
    print(s)

임호태 바보 바라워 그렇게 살아 바보 빨리 얘기하자 명국이 이기록 늘 읽을어 임화 달라고</s>이정도 정리할레 희망을 계속 자영 이을근 찬양군 사용 요구냉장 같은 태도 못할 말북 상업 사용할 엄만은 답답망부족 기능뚱 두 전화번호 미차고 무기 내일 갈 휴무50 며칠 어차피 술을 마흐트러 임문이 30 일주일 허리 벗어 밤이 무한 숨벅 천안 정화대로 미쳐 주말 빛고추 주인이 옮기서 밝 혀 돌아오셔야 옹넝 한계탄소주택 여섯무록 온다 길어 배상 견인님 조상!</s>낙하로 ▷현석 보호방싱 사업자 헥인기운 포함킬거나 쓰러져손 따서 되면 예고탬 쾌인용 집단복구《편한친 삽추는 비밀입찰씬비에 몰라도 풍산 도로 조망 어려움맵音멘法 협력 가능성은 심사위원껏캄쉘 술 지는 숫자 훼인하 근처 환상분양가나 페이퍼 트래픽 USB@nerfor2013 본격조명 갓공항무진집은 보일옛 모양이 참북찍하다는 저무기 굳텃 짐 차에 플러그방 은 값이 700외종의 뜻 탄력태양광 권사들이 보일러 펼 때다 북
